# Solovay-Kitaev theorem 

### Basic Approximation
Implement Basic Approximation using a KDree as an efficient lookup table for n-dimensional data. This gives the initial basic approximation. 

In [2]:
import numpy as np
from sklearn.neighbors import KDTree
from itertools import product

def operator_norm(A):
    return np.linalg.norm(A, ord=2)

def to1D(arr):
    return np.hstack( (arr.real.flatten(), arr.imag.flatten()) )

def gen_basic_lookup(basis, max_depth): 
    # create basic lookup routine by storing sequences of gate sets in a KDtree 
    seqs, approxs = list(basis.keys()), list(basis.values())
    
    # the multiplication of sequence is like a tree expansion
    seqs_alldepth = seqs.copy()
    approxs_alldepth = approxs.copy() 
    for i in range(max_depth-1):
        seqs = [seq + syb for seq in seqs for syb in basis.keys()]
        approxs = [np.matmul(a, g)  for a in approxs for g in basis.values()]    
        seqs_alldepth += seqs 
        approxs_alldepth += approxs
    return seqs_alldepth, approxs_alldepth



### Test Basic Approximation

In [6]:
# H and T gate as basis (hardware gat set)
basis = {"H": 1/np.sqrt(2.0) * np.array([[1, 1],[1, -1]]),"T" : np.array([[1, 0], [0, np.exp(1j*np.pi/4)]])}

# Generate a basic lookup sequence
seqs, approxs = gen_basic_lookup(basis, 10)
formatted_approxs = np.array([to1D(arr) for arr in approxs])
tree = KDTree(formatted_approxs)

H = 1/np.sqrt(2.0) * np.array([[1, 1],[1, -1]])
T = np.array([[1, 0], [0, np.exp(1j*np.pi/4)]])

# Build target matrix U = HT, and see if the returned sequence and corresponding matrix is HT
dist, index = tree.query( to1D(np.matmul(H,T)).reshape(1, -1) , k=1)

print(   seqs[index[0][0]])
print(approxs[index[0][0]])

HT
[[ 0.70710678+0.j   0.5       +0.5j]
 [ 0.70710678+0.j  -0.5       -0.5j]]


### Group Commutator Decomposition

In [8]:

# Helper functions for group commutator decomposition 
def rx(phi):
    """Single-qubit rotation for operator sigmax with angle phi."""
    return np.array([[np.cos(phi / 2), -1j * np.sin(phi / 2)],
                     [-1j * np.sin(phi / 2), np.cos(phi / 2)]])

def ry(phi):
    """Single-qubit rotation for operator sigmay with angle phi."""
    return np.array([[np.cos(phi / 2), -np.sin(phi / 2)],
                     [np.sin(phi / 2), np.cos(phi / 2)]])

def rz(phi):
    """Single-qubit rotation for operator sigmaz with angle phi."""
    return np.arrayj([[np.exp(-1j * phi / 2), 0],
                     [0, np.exp(1j * phi / 2)]])

# Get unitary axis and angle
def bloch(U):
#     if isinstance(U, qt.Qobj):
#         U = U.full()
    angle = np.real(2 * np.arccos(np.trace(U) / 2))
    sin = np.sin(angle / 2)
    eps = 1e-10
    if sin < eps:
        axis = [0, 0, 1]
    else:
        nz = np.imag(U[1, 1] - U[0, 0]) / (2 * sin)
        nx = -np.imag(U[1, 0]) / sin
        ny = np.real(U[1, 0]) / sin
        axis = [nx, ny, nz]
    return axis, angle

def diagonalize(A):
    d, V = np.linalg.eig(A)
    return d, V

def gcd(U):
    #     print('---', U.isunitary)
    theta = 2 * np.arccos(np.real(np.trace(U) / 2))
    phi = 2 * np.arcsin(  np.sqrt(np.sqrt((0.5 - 0.5 * np.cos(theta / 2)))))
    axis, angle = bloch(U)
    V = rx(phi)    
    if axis[2] < 0:
        W = ry(2 * np.pi - phi)
    else:
        W = ry(phi)
        
    _, V1 = diagonalize(U)
    _, V2 = diagonalize(np.matmul(np.matmul(V,W), np.matmul(V.conj().T , W.conj().T)) )
    
    S = np.matmul(V1 , V2.conj().T)
    V_tilde = np.matmul(S, np.matmul(V, S.conj().T))
    W_tilde = np.matmul(S, np.matmul(W, S.conj().T))
    
    return V_tilde, W_tilde

In [25]:
def check_isunitary(self):
        """
        Checks whether qobj is a unitary matrix
        """
        if self.isoper:
            eye_data = fast_identity(self.shape[0])
            return not (
                np.any(
                    np.abs((self.data*self.dag().data - eye_data).data)
                    > settings.atol
                )
                or
                np.any(
                    np.abs((self.dag().data*self.data - eye_data).data)
                    > settings.atol
                )
            )
        else:
            return False

### Solovay-Kitaev 
Putting everything together, we have the Solovay-Kitaev algorithm

In [10]:
# inputs 
# l_0 is the initial approx seq length
l_0 = 10
 
# basis 
basis = {
    "H": 1/np.sqrt(2.0) * np.array([[1, 1],[1, -1]]),
    "T" : np.array([[1, 0], [0, np.exp(1j*np.pi/4)]])}

seqs, approxs = gen_basic_lookup(basis, l_0)
formatted_approxs = np.array([to1D(arr) for arr in approxs])
tree = KDTree(formatted_approxs)

def Solovay_Kitaev(U, n):
    if n == 0:
        _, index = tree.query(to1D(U).reshape(1, -1), k=1)        
        return approxs[index[0][0]], seqs[index[0][0]]   
    else: 
        U1, seqU1 = Solovay_Kitaev(U, n-1)
        V, W = gcd(np.matmul(U, U1.conj().T)) 
        V1, seqV1 = Solovay_Kitaev(V, n-1)
        W1, seqW1 = Solovay_Kitaev(W, n-1)
        Vd1 = V1.conj().T
        Wd1 = W1.conj().T
        seqVd1 = '(' + seqV1 + ')'
        seqWd1 = '(' + seqW1 + ')'
        approx = np.matmul(np.matmul(np.matmul(V1, W1), np.matmul(Vd1, Wd1)), U1)
        seq = seqV1 + seqW1 + seqVd1 + seqWd1 + seqU1
    return approx, seq

In [11]:
from scipy.stats import unitary_group
# random SU(2) as the target Unitary gate to be approximated
targetGate = unitary_group.rvs(2) 
approx, seq = Solovay_Kitaev(targetGate , 1)
d = targetGate - approx

In [12]:
print("Compiled sequence: ", seq)
print("Difference between approx and target Unitary gate", d)


Compiled sequence:  HTHTTTTTTTHTTTTHTTTT(HTHTTTTTTT)(HTTTTHTTTT)HTTTTHTTTT
Difference between approx and target Unitary gate [[-0.51254509+0.49251119j -0.27125718+0.32974248j]
 [ 0.35425893+0.79361184j -0.43930565-0.7641384j ]]


### Algorithm Reference: 
["The Solovay-Kitaev algorithm"](https://arxiv.org/abs/quant-ph/0505030), Christopher M. Dawson, Michael A. Nielsen. 

### Implementation Reference: 
- [Simple example using QuTip in python by PQCLab](https://github.com/PQCLab/SolovayKitaev) 
- [Python by cryptogoth](https://github.com/cryptogoth/skc-python/blob/master/skc/kdtree.py): Well commented code but many lines 
- [in Python by ssmi1975](https://github.com/ssmi1975/solovay-kitaev-py): functional style, 1000 lines, a bit hard to parse, has nice plots.
- [Official Implementation by Dawson in C++](https://github.com/cmdawson/sk)
- [Viz and numerical analysis by sesajad](https://github.com/sesajad/solovay-kitaev-simulation)
- [Qiskit Transpiler Plugin](https://qiskit.org/documentation/stubs/qiskit.transpiler.passes.SolovayKitaevSynthesis.html)